In [31]:
from PIL import Image, ImageFilter

import numpy as np

im = Image.open('Yugra.png').convert('RGB')

im.show()

In [15]:
values = np.array(im)
print(values.shape)

(222, 350, 3)


In [24]:
palette = np.array([[0, 0, 0],
                    [255, 255, 255]
                    ])

# k = 31
# lin = np.linspace(0, k**3 - 1, num=k**3)

# def color(x):
#     ones = x % k
#     tens = (x // k) % k
#     hundreds = ((x / k) // k)
    
#     return [0 if ones == 0 else ones * (256//(k - 1)) - 1, 0 if tens == 0 else tens * (256//(k - 1)) - 1, 0 if hundreds == 0 else hundreds * (256//(k - 1)) - 1]

# palette = np.array([color(x) for x in lin])

In [45]:
def recolor_manhattan(image, palette):
    x, y, _ = image.shape
    recolored = np.zeros((x, y, 3))
    for i in range(x):
        for j in range(y):
            argcolor = np.sum(np.abs(palette - image[i,j]), axis=1).argmin()
            recolored[i,j] = palette[argcolor]
    return recolored

In [27]:
recolored_manhattan_np = recolor_manhattan(values, palette)

recolored_manhattan = Image.fromarray(recolored_manhattan_np.astype(np.uint8))

In [28]:
recolored_manhattan.show()

In [6]:
def recolor_euclidean(image, palette):
    x, y, _ = image.shape
    recolored = np.zeros((x, y, 3))
    for i in range(x):
        for j in range(y):
            argcolor = np.sum(np.abs(palette - image[i,j])**2, axis=1).argmin()
            recolored[i,j] = palette[argcolor]
    return recolored

In [25]:
recolored_euclidean_np = recolor_euclidean(values, palette)

recolored_euclidean = Image.fromarray(recolored_euclidean_np.astype(np.uint8))

In [26]:
recolored_euclidean.show()

In [11]:
def convolve(img, filt):
    dim = np.ndim(img)
    convolved_img = np.zeros(img.shape)
    if(np.ndim(filt) == 1):
        l = filt.shape[0]
        filt = np.vstack(((np.zeros(l)), filt, np.zeros(l)))
        k = 3
    else:
        l, k = filt.shape
    hdiff = (l - 1) // 2
    vdiff = (k - 1) // 2 
    convolution = np.flipud(filt)
    convolution = np.fliplr(convolution)    
    if dim == 3:
        m, n, d = img.shape
        paddedimg = np.zeros((m + 2 * hdiff, n + 2 * vdiff))
        for i in range(d):
            paddedimg[:, :] = np.pad(img[:, :, i], ((hdiff, hdiff), (vdiff, vdiff)),
                         mode='constant')
            for x in range(m):
                for y in range(n):
                    subsection = paddedimg[x: x + 2 * hdiff + 1, y: y + 2 * vdiff + 1]
                    val = np.sum(subsection * convolution)
                    convolved_img[x, y, i] = val
    else: 
        m, n = img.shape
        paddedimg = np.zeros((m + 2 * hdiff, n + 2 * vdiff))
        paddedimg = np.pad(img, ((hdiff, hdiff), (vdiff, vdiff)),
                     mode='constant')
        for x in range(m):
            for y in range(n):
                subsection = paddedimg[x: x + 2 * hdiff + 1, y: y + 2 * vdiff + 1]
                val = np.sum(subsection * convolution)
                convolved_img[x, y] = val

    return convolved_img

def gaussian_filter(k, sigma):
    arr = np.linspace(-(k - 1) / 2., (k - 1) / 2., k)
    gauss = np.exp(-0.5 * np.square(arr / sigma))
    filt = np.outer(gauss, gauss)
    return filt / np.sum(filt)

In [29]:
filt = gaussian_filter(5, 2)
filtered = convolve(values, filt)
filtered_euclidean = recolor_euclidean(filtered, palette)
filtered_img = Image.fromarray(filtered_euclidean.astype(np.uint8))

In [30]:
filtered_img.show()